In [2]:
import pandas as pd
from wmfdata import hive

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


## New editor retention

In [ ]:
gs_ner = wmf.hive.run(
"""
with gs_new_edits as (
    select 
        gd.wiki_db,
        event_user_text as user_name,
        event_timestamp as edit_dt,
        event_user_creation_timestamp as registration_dt
    from wmf.geoeditors_daily gd
    left join canonical_data.countries cdc
    on gd.country_code = cdc.iso_code
    left join wmf.mediawiki_history mh
    on
        gd.wiki_db = mh.wiki_db and
        gd.user_fingerprint_or_id = mh.user_id and
        mh.snapshot = "{snapshot}"
    where
        month >= "{start}" and
        economic_region = "Global South" and
        not event_user_is_anonymous and
        not event_user_is_created_by_system and
        not (event_user_is_bot_by_name or array_contains(event_user_groups, "bot")) and
        event_user_creation_timestamp >= "{start}" and
        event_user_creation_timestamp < "{end}"
)
select 
    1st_month.cohort as cohort,
    sum(cast(1st_month.edits >= 1 as int)) as new_editors,
    sum(cast(2nd_month.edits >= 1 as int)) / sum(cast(1st_month.edits >= 1 as int)) as new_editor_retention
from (
    select
        user_name,
        wiki_db as wiki,
        substr(registration_dt, 0, 7) as cohort,
        count(*) as edits
    from gs_new_edits
    where
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") <
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60))
    group by user_name, registration_dt, wiki_db
    ) 1st_month
left join (
    select
        user_name,
        wiki_db as wiki,
        substr(registration_dt, 0, 7) as cohort,
        count(*) as edits
    from gs_new_edits
    where
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") >=
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60)) and
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") <
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (60*24*60*60))
        group by user_name, registration_dt, wiki_db
    ) 2nd_month
on
    (1st_month.user_name = 2nd_month.user_name and
    1st_month.wiki = 2nd_month.wiki and
    1st_month.cohort = 2nd_month.cohort)
group by 1st_month.cohort
""".format(
    snapshot="2018-10",
    start="2018-05",
    end="2018-09"
))

gs_ner

In [15]:
gs_ner

,cohort,new_editors,new_editor_retention
0,2018-05,2108,0.008065
1,2018-06,2747,0.012741
2,2018-07,4627,0.009942
3,2018-08,37810,0.001455
